# OHW 2021 - Model Subsampling Project

## OceanHackWeek21 project to subsample high-resolution model output as if by gliders, ships, or other *in situ* platforms

The goal of this project is to create a Python package that takes an input trajectory (e.g., the path of an ocean glider), subsamples output from a high-resolution ocean simulation along that trajectory, and returns a set of subsampled variables (e.g., standard physical variables temperature, salinity, velocity; derived physical quantities such as steric height; biogeochemical quantities if available). We envision this package having two potential uses: 1) designing in situ sampling strategies, and 2) interpreting in situ data in the context of a highly resolved oceanographic model.

In [33]:
## Imports

# Third-party packages for data manipulation
import numpy as np
import pandas as pd
import xarray as xr

# Third-party packages for data interpolation
from scipy import interpolate

# Third-party packages for data visualizations
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline

### Import model data

We chose to subsample [LLC4320](https://podaac.jpl.nasa.gov/datasetlist?ids=Processing+Levels&values=4+-+Gridded+Model+Output&search=Pre-SWOT+llc4320&view=list&provider=). [DOES ANYONE WANT TO SAY MORE ABOUT THIS?]. The data from the model was retrieved using the first section of `Pre-SWOT_Numerical_Simulation_Demo.ipynb` (everything before the "Plot eight 2D fields" heading) and saved as `LLC4320_pre-SWOT_ACC_SMST_20111221.nc`. Here, we just load the data file. 

In [34]:
## Load model data

ds = xr.open_dataset('LLC4320_pre-SWOT_ACC_SMST_20111221.nc')

### If you would like to subsample the model using a real glider track, run the following cells.

Our example glider track original came from [this repo](https://github.com/earthcube2021/ec21_balwada_etal). It wasn't inside one of the regions covered by LLC4320. Here, we load the trajectory and transpose it such that it fits within the Southern Ocean region.

In [35]:
## Load and transpose glider data

# Load data
ds_CTD_659 = xr.load_dataset('CTD_659.nc')

# Print glider boundaries
s = 'Original glider boundaries:\nNorth: {n}\nSouth: {s}\nEast: {e}\nWest: {w}\n'.format(
    n=ds_CTD_659.latitude.data.max(),
    s=ds_CTD_659.latitude.data.min(),
    e=ds_CTD_659.longitude.data.max(),
    w=ds_CTD_659.longitude.data.min()
)
print(s)

# Get boundaries of model region
model_boundary_n = ds.YC.max()
model_boundary_s = ds.YC.min()
model_boundary_w = ds.XC.min()
model_boundary_e = ds.XC.max()

# Print model boundaries
s = 'Model boundaries:\nNorth: {n}\nSouth: {s}\nEast: {e}\nWest: {w}\n'.format(
    n=model_boundary_n.data,
    s=model_boundary_s.data,
    e=model_boundary_e.data,
    w=model_boundary_w.data
)
print(s)

# Transpose latitude
shifted_lat = (ds_CTD_659.latitude - ds_CTD_659.latitude.min()
              )/(ds_CTD_659.latitude.max() - ds_CTD_659.latitude.min()
                )*(model_boundary_n-model_boundary_s)+ model_boundary_s


# Transpose longitude
shifted_lon = (ds_CTD_659.longitude - ds_CTD_659.longitude.min()
              )/(ds_CTD_659.longitude.max() - ds_CTD_659.longitude.min()
                )*(model_boundary_e-model_boundary_w)+ model_boundary_w

# Print transposed glider boundaries
s = 'Transposed glider boundaries:\nNorth: {n}\nSouth: {s}\nEast: {e}\nWest: {w}\n'.format(
    n=shifted_lat.max().data,
    s=shifted_lat.min().data,
    e=shifted_lon.max().data,
    w=shifted_lon.min().data
)
print(s)

Original glider boundaries:
North: -50.363265
South: -53.458083333333335
East: 38.99485
West: 30.026965

Model boundaries:
North: -53.00566864013672
South: -56.989952087402344
East: 154.28125
West: 150.3020782470703

Transposed glider boundaries:
North: -53.00566864013672
South: -56.989952087402344
East: 154.28125
West: 150.3020782470703



LLC4320 data does not have lat, lon as coordinates; instead the coordinates are simply index numbers. This means that the lon-lat-depth passed for the trajectory will have to be converted to the corresponding index numbers.

In [36]:
## Convert lat, lon to index

X = ds.XC
Y = ds.YC
i = ds.i
j = ds.j
z = ds.Z
k = ds.k

f_x = interpolate.interp1d(X[0,:].values, i)
f_y = interpolate.interp1d(Y[:,0].values, j)
f_z = interpolate.interp1d(z, k, bounds_error=False)

In [39]:
## Assemble trajectory dataset

# Remove NaN values from pressure (depth) data
depth = -ds_CTD_659.pressure.where(~np.isnan(ds_CTD_659.pressure), drop=True)
n = len(depth)

# Assemble dataset
survey_track = xr.Dataset(
    dict(
        i = xr.DataArray(f_x(shifted_lon.where(~np.isnan(ds_CTD_659.pressure), drop=True)), dims='points'),
        j = xr.DataArray(f_y(shifted_lat.where(~np.isnan(ds_CTD_659.pressure), drop=True)), dims='points'),
        k = xr.DataArray(f_z(depth), dims='points'),
        time = xr.DataArray(np.linspace(ds.time[0], ds.time[1], num=n),dims='points'),
    )
)

UFuncTypeError: ufunc 'multiply' cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [40]:
ds.time.data[0]

numpy.datetime64('2011-12-21T00:00:00.000000000')